For the visualization of the people-event network from the CESTA events data
    1. csv of events with descriptions
    2. each event will become a node (JSON)
        event-node
        - title: event title from the csv
        - info: event info from the csv
        - link: also from the csv
        - color: different color for each year -> https://identity.stanford.edu/design-elements/color/accent-colors/
        - size: smaller than people
    3. NER on the event data
    4. For each event, identify the unique people in the description
    5. each person will become a node
        person-node
        - title: person name
        - info: event titles and links that the person is mentioned in
        - color: same color for all persons, using a compatible color from the palette as the event-nodes
        - size: growing with number of events
    6. Color of the link will be grey
    7. Layout of the graph will be force-directed
    8. We will display people names on nodes but not event names 



In [ ]:
import os
import re
import pandas as pd
import csv
import numpy as np

In [ ]:
# Reformat the events csv for the event-node